In [2]:
import translators as ts
import translators.server as tss
from presidio_analyzer import AnalyzerEngine, Pattern
from presidio_anonymizer import AnonymizerEngine
from presidio_anonymizer.entities import OperatorResult, OperatorConfig
from presidio_analyzer import PatternRecognizer
import random

Using region Central Macedonia server backend.


In [29]:
class Tran:
    def __init__(self, lang, text):
        self.text = text
        self.lang = lang

        #List of entities to look for
        self.entities = ['PERSON', 'NUMBERS',
                   'EMAIL_ADDRESS', 'IP_ADDRESS', 'NRP',
                   'LOCATION', 'BLOOD_TYPE']
        
        self.analyzer = AnalyzerEngine(log_decision_process=True)
        self.anonymizer = AnonymizerEngine()
        
        # Option to create custom recognizers
        blood_type_recognizer = PatternRecognizer(supported_entity="BLOOD_TYPE",
                                      deny_list=["A-","A+","B-","B+","AB-","AB+","O-","O+"])
        
        numbers_pattern = Pattern(name="numbers_pattern",regex="\d+", score = 0.3)

        number_recognizer = PatternRecognizer(supported_entity="NUMBERS", patterns = [numbers_pattern])

        self.analyzer.registry.add_recognizer(blood_type_recognizer)
        self.analyzer.registry.add_recognizer(number_recognizer)

        # Can define how the operators will behave for each entity
        self.operators = {
            "DEFAULT": OperatorConfig("replace", {"new_value": "<ANONYMIZED>"}),
            
            "PERSON": OperatorConfig("replace", {"new_value": "<ANONYMOUS>"}),
            
            "NUMBERS": OperatorConfig("replace", {"new_value": "<ANONYMIZED>"}),
            
            "IP_ADDRESS": OperatorConfig("mask", {"type": "mask","masking_char": "*",
                                                    "chars_to_mask": 12,
                                                    "from_end": True,}),
            "NRP": OperatorConfig("redact"),
            
            "LOCATION": OperatorConfig("replace", {"new_value": "<HIDDEN>"}),
            
            "EMAIL_ADDRESS": OperatorConfig("replace", {"new_value": "<HIDDEN_EMAIL>"}),
            
            "BLOOD_TYPE": OperatorConfig("replace", {"new_value": "<HIDDEN_BLOOD_TYPE>"})
        
        }
    
    def translate(self):
        #translates from greek to english 
        self.translated_txt = ts.translate_text(self.text, translater='google',
                                            from_language='el',
                                            to_language='en')
        return self.translated_txt


    def anonymize(self):
                
        if self.lang == 'el':
        # Anonymize the translated text
            self.results = self.analyzer.analyze(text=self.translated_txt,
                                                entities=self.entities,
                                                language='en')
            self.anon_text = self.anonymizer.anonymize(text=self.translated_txt, analyzer_results=self.results,
                                                operators=self.operators)
            return self.anon_text
        
        #Anonymize english text
        elif self.lang == 'en':
            self.results = self.analyzer.analyze(text=self.text,
                                                entities=self.entities,
                                                language='en')
            self.anon_text = self.anonymizer.anonymize(text=self.text, analyzer_results=self.results,
                                                operators=self.operators)
            return self.anon_text
        
    def PII_removal_from_original(self):
        start = self.results[0].to_dict().get('start')
        end = self.results[0].to_dict().get('end')

        if self.lang == 'el':
            pii_translated = ts.translate_text(self.translated_txt[start:end], translater='google',
                                           from_language='en',to_language='el')
        
            return self.text.replace(pii_translated, self.anon_text.items[0].text)
        
        if self.lang == 'en':
            return self.anon_text.text



In [4]:
def process_user_input():

    languages = ['el','en']
    exit_phrases = ['goodbye','bye','good bye','see you']
    chatbot_prompts = ['What is your name?', 'I need your credit card number to make this purchase',
                       'What email address should I send this to?', 'Tell me about yourself',
                       'What number should they call you at?', 'Where do you live?',
                       'What is your blood type?','What IP address are you connected from?']

    lang_input = str(input('Choose language: \n\
    1) Greek \n\
    2) English \n'))
    if lang_input.lower() == 'greek' or lang_input == "1":
        chosen_lang = languages[0]
    elif lang_input.lower() == 'english' or lang_input == "2":
        chosen_lang = languages[1]
    else:
        print('Language not supported')

    while True:
        global user_tran
        user_input = str(input(f'Say something personal: '))
        user_tran = Tran(chosen_lang, user_input)
        print(f'This is text: {user_tran.text}')

        if chosen_lang == 'el':
            user_tran.translate()

        if user_tran.text.lower() in exit_phrases:
            print('Goodbye!')
            break

        user_tran.anonymize()
        print(user_tran.results)
        print(user_tran.text)
        print(user_tran.anon_text)
        
        print(user_tran.PII_removal_from_original())



In [30]:
text = 'με λενε γιαννη και ειμαι 18 ετων. μενω στο Ηράκλειο'

test = Tran('el', text)

In [32]:
test.translate()


'My name is Yiannis and I am 18 years old. I live in Heraklion'

In [45]:
test.analyzer.analyze(text=test.translate(),language='en')

[2023-07-03 19:16:45,072][decision_process][INFO][None][nlp artifacts:{"entities": ["Yiannis", "18 years old", "Heraklion"], "tokens": ["My", "name", "is", "Yiannis", "and", "I", "am", "18", "years", "old", ".", "I", "live", "in", "Heraklion"], "lemmas": ["my", "name", "be", "Yiannis", "and", "I", "be", "18", "year", "old", ".", "I", "live", "in", "Heraklion"], "tokens_indices": [0, 3, 8, 11, 19, 23, 25, 28, 31, 37, 40, 42, 44, 49, 52], "keywords": ["yiannis", "18", "year", "old", "live", "heraklion"]}]
[2023-07-03 19:16:45,076][decision_process][INFO][None][["{'entity_type': 'LOCATION', 'start': 52, 'end': 61, 'score': 0.85, 'analysis_explanation': {'recognizer': 'SpacyRecognizer', 'pattern_name': None, 'pattern': None, 'original_score': 0.85, 'score': 0.85, 'textual_explanation': \"Identified as GPE by Spacy's Named Entity Recognition\", 'score_context_improvement': 0, 'supportive_context_word': '', 'validation_result': None}, 'recognition_metadata': {'recognizer_name': 'SpacyRecogni

[type: PERSON, start: 11, end: 18, score: 0.85,
 type: DATE_TIME, start: 28, end: 40, score: 0.85,
 type: LOCATION, start: 52, end: 61, score: 0.85,
 type: NUMBERS, start: 28, end: 30, score: 0.3]

In [57]:
test.anonymizer.logger.callHandlers()

TypeError: Logger.callHandlers() missing 1 required positional argument: 'record'

In [ ]:
import logging
